In [1]:
import pandas as pd
import numpy as np

### Electree Class

In [2]:
class Electree:
    
    def __init__(self, alternatives, boundries, boundries_thresholds, types_df):
        self.df_alternatives = alternatives
        self.df_boundries = boundries
        self.df_thresholds = boundries_thresholds
        self.types = types_df["type"]
        self.names = alternatives.index
        self.weights = types_df["weights"]

        ps = self.df_thresholds.loc[:, (slice(None), "p")].to_numpy()[np.newaxis, ...]
        qs = self.df_thresholds.loc[:, (slice(None), "q")].to_numpy()[np.newaxis, ...]
        vs = self.df_thresholds.loc[:, (slice(None), "v")].to_numpy()[np.newaxis, ...]

        self.pqs = np.concatenate([ps, qs])
        self.pvs = np.concatenate([ps, vs])

    
    def marginal_concordances_vectorized_v2(self, alternatives, np_p_q, boundries, types, ab=True):
        ## I belive that now it is far more flexible and it should not consume that much memory
        ## because now we do not create those huge matrices(relatively huge) but we simply make use of numpy's broadcast capabilities
        
        
        # alternatives = np.tile(alternatives.to_numpy(), (4,1)).reshape((4, 17, 8))
        
        # boundries_matrix = np.tile(boundries.to_numpy(), (1, alternatives.shape[1])).reshape((4, 17, 8))

        # end_result = np.zeros(shape=alternatives.shape)

        p = np_p_q[0]
        q = np_p_q[1]

        # p_vectorized = np.tile(p, (1, alternatives.shape[1])).reshape((4, 17, 8))
        # q_vectorized = np.tile(q, (1, alternatives.shape[1])).reshape((4, 17, 8))

        ##number of boundries
        nob = len(boundries)
        ##number of alternatives
        noa = len(alternatives)
        ##number of functions
        nof = len(p[0])
        if ab:
            alternatives = alternatives.to_numpy()[np.newaxis, ...]
            boundries_matrix = boundries.to_numpy()[:, np.newaxis, :]
        else:
            boundries_matrix = alternatives.to_numpy()[np.newaxis, :, :]
            alternatives = boundries.to_numpy()[:, np.newaxis, :]
            
        
        p_vectorized = p[:, np.newaxis, :]
        q_vectorized = q[:, np.newaxis, :]

        
        end_result = np.zeros(shape=(nob, noa, nof))

        
    
        for function_type in ("gain", "cost"):
            gain_type = types == function_type

            test = end_result[..., np.where(gain_type)[0]]

            

            
            if function_type == "gain":
                # print(gain_type.shape, alternatives.shape, q_vectorized.shape)
                op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) >= -q_vectorized[..., gain_type]
                op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <  -p_vectorized[..., gain_type]
            else:
                op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <= q_vectorized[..., gain_type]
                op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) > p_vectorized[..., gain_type]
            
            op3 = ~op1 & ~op2
            
            
            test[op1] = 1
            test[op2] = 0
            
            if function_type == "gain":
                test[op3] = ( ( p_vectorized[..., gain_type] - (boundries_matrix[..., gain_type] - alternatives[..., gain_type]) ) / (p_vectorized[..., gain_type] - q_vectorized[..., gain_type]) )[op3]
            else:
                test[op3] = ( ( p_vectorized[..., gain_type] - (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) ) / (p_vectorized[..., gain_type] - q_vectorized[..., gain_type]) )[op3]
            
            end_result[..., np.where(gain_type)[0]] = test
        
        return end_result


    def marginal_discordances_vectorized(self, alternatives, n_p_v, boundries, types, ab=True):
        ## I belive that now it is far more flexible and it should not consume that much memory
        ## because now we do not create those huge matrices(relatively huge) but we simply make use of numpy's broadcast capabilities
        
        
        # alternatives = np.tile(alternatives.to_numpy(), (4,1)).reshape((4, 17, 8))
        
        # boundries_matrix = np.tile(boundries.to_numpy(), (1, alternatives.shape[1])).reshape((4, 17, 8))

        # end_result = np.zeros(shape=alternatives.shape)

        p = n_p_v[0]
        v = n_p_v[1]

        # p_vectorized = np.tile(p, (1, alternatives.shape[1])).reshape((4, 17, 8))
        # q_vectorized = np.tile(q, (1, alternatives.shape[1])).reshape((4, 17, 8))

        ##number of boundries
        nob = len(boundries)
        ##number of alternatives
        noa = len(alternatives)
        ##number of functions
        nof = len(p[0])
        if ab:
            alternatives = alternatives.to_numpy()[np.newaxis, ...]
            boundries_matrix = boundries.to_numpy()[:, np.newaxis, :]
        else:
            boundries_matrix = alternatives.to_numpy()[np.newaxis, :, :]
            alternatives = boundries.to_numpy()[:, np.newaxis, :]
            
        
        p_vectorized = p[:, np.newaxis, :]
        v_vectorized = v[:, np.newaxis, :]

        
        end_result = np.zeros(shape=(nob, noa, nof))

        
    
        for function_type in ("gain", "cost"):
            gain_type = types == function_type

            test = end_result[..., np.where(gain_type)[0]]

            

            
            if function_type == "gain":
                # print(gain_type.shape, alternatives.shape, q_vectorized.shape)
                op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <= -v_vectorized[..., gain_type]
                op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) >= -p_vectorized[..., gain_type]
            else:
                op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) >= v_vectorized[..., gain_type]
                op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <= p_vectorized[..., gain_type]
            
            op3 = ~op1 & ~op2
            
            
            test[op1] = 1
            test[op2] = 0
            
            if function_type == "gain":
                test[op3] = ( ( (boundries_matrix[..., gain_type] - alternatives[..., gain_type]) - p_vectorized[..., gain_type] ) / (v_vectorized[..., gain_type] - p_vectorized[..., gain_type]) )[op3]
            else:
                test[op3] = ( ( v_vectorized[..., gain_type] - (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) ) / (v_vectorized[..., gain_type] - p_vectorized[..., gain_type]) )[op3]
            
            end_result[..., np.where(gain_type)[0]] = test
        
        return end_result
    
    def outranking_credibility(self, marginal_concordances_tensor, marginal_discordances_tensor, weights):
        weights = np.array(weights)
        final_concordances = (marginal_concordances_tensor * (weights / weights.sum())).sum(axis=2)[..., np.newaxis]
        products = marginal_discordances_tensor > final_concordances

        
        test = np.zeros(marginal_discordances_tensor.shape)
        test1 = np.ones(marginal_discordances_tensor.shape)
        
        test[products] = marginal_discordances_tensor[products]
        
        test1[products] = ( (1 - test) / (1 - final_concordances) )[products]
        test2 = test1.prod(axis=2)[..., np.newaxis]
        
        
        return test2
        
        
    def create_symbols(self, as_outranking, bs_outranking, sigma=0.65):

        concatenated = np.concatenate([as_outranking, bs_outranking], axis=2)

        utility = np.where(concatenated >= sigma, 1, 0)
        
        signs = np.empty(shape=concatenated.shape[:-1], dtype=str)

        boolean_better = utility[:, :, 0] > utility[:, :, 1]
        boolean_worse = utility[:, :, 0] < utility[:, :, 1]
        boolean_indifferent = (utility[:, :, 0] == utility[:, :, 1]) & (utility[:, :, 0] == 1)
        boolean_incomparable = (utility[:, :, 0] == utility[:, :, 1]) & (utility[:, :, 0] == 0)

        signs[boolean_better] = ">"
        signs[boolean_worse] = "<"
        signs[boolean_indifferent] = "/"
        signs[boolean_incomparable] = "?"
        
        return signs.T


    def optimistic(self, signs):
        return (pd.DataFrame(data=signs).isin((">", "/", "?")).sum(axis=1) + 1).to_frame().set_index(self.names).rename({0: "Class"}, axis=1)
    def not_optimistic(self, signs):
        return ( (~pd.DataFrame(data=signs).isin(("<", "?"))).sum(axis=1) + 1 ).to_frame().set_index(self.names).rename({0: "Class"}, axis=1)

    def glue_it(self):
        if hasattr(self, "symbols"):
            return self.symbols

        alternatives = self.df_alternatives
        boundries = self.df_boundries
        

        as_concordances = self.marginal_concordances_vectorized_v2(alternatives, self.pqs, boundries, self.types)
        bs_concordances = self.marginal_concordances_vectorized_v2(alternatives, self.pqs, boundries, self.types, ab=False)

        as_discordances = self.marginal_discordances_vectorized(alternatives, self.pvs, boundries, self.types)
        bs_discordances = self.marginal_discordances_vectorized(alternatives, self.pvs, boundries, self.types, ab=False)

        # weights = [ 5,  6,  3,  4,  9, 10, 10,  4]
        as_outranking = self.outranking_credibility(as_concordances, as_discordances, self.weights)
        bs_outranking = self.outranking_credibility(bs_concordances, bs_discordances, self.weights)

        symbols = self.create_symbols(as_outranking, bs_outranking)
        
        self.symbols = symbols

        return symbols
    


        
    

In [3]:
alternatives = pd.read_csv("data/Alternatives_correction.csv", index_col="Name")

In [4]:
boundries = pd.read_csv("data/boundries.csv", index_col="Boundries")

In [5]:
thresholds = pd.read_csv("data/thresholds.csv", index_col=0, header=[0, 1])

In [6]:
test_df_inner = pd.read_csv("data/p_q_correction.csv", index_col="Criteria")

In [7]:
ele = Electree(alternatives, boundries, thresholds, test_df_inner)

In [8]:
symbols = ele.glue_it()

/tmp/ipykernel_44091/3063971603.py:171: RuntimeWarning: divide by zero encountered in true_divide
  test1[products] = ( (1 - test) / (1 - final_concordances) )[products]


In [9]:
ele.not_optimistic(symbols)

,Class
Name,
Demon's souls,3
Dark Souls I,3
Dark Souls II,3
Dark Souls III,2
Dark Souls III: Ashes of Ariandel,2
Dark Souls III: The Ringed City,2
Bloodborne,4
Sekiro: Shadows Die Twice,1
Elden Ring,1


In [10]:
ele.optimistic(symbols)

,Class
Name,
Demon's souls,4
Dark Souls I,5
Dark Souls II,4
Dark Souls III,5
Dark Souls III: Ashes of Ariandel,5
Dark Souls III: The Ringed City,5
Bloodborne,5
Sekiro: Shadows Die Twice,5
Elden Ring,5


In [11]:
(ele.optimistic(symbols) - ele.not_optimistic(symbols)).rename({"Class": "Optimisitc-Pessimistic difference"}, axis=1)

,Optimisitc-Pessimistic difference
Name,
Demon's souls,1
Dark Souls I,2
Dark Souls II,1
Dark Souls III,3
Dark Souls III: Ashes of Ariandel,3
Dark Souls III: The Ringed City,3
Bloodborne,1
Sekiro: Shadows Die Twice,4
Elden Ring,4


### BattleGround

In [12]:
alternatives = pd.read_csv("data/Alternatives_correction.csv", index_col="Name")
alternatives.head(20)

,Price,Difficulty,Playthrough time,Graphics,Lore,Soundtrack,Gameplay,Bugs
Name,,,,,,,,
Demon's souls,50,71,40,58,92,69,81,34
Dark Souls I,60,77,40,81,100,89,68,32
Dark Souls II,70,62,50,67,70,79,56,62
Dark Souls III,150,90,60,86,89,95,94,20
Dark Souls III: Ashes of Ariandel,50,100,15,85,100,97,91,18
Dark Souls III: The Ringed City,50,100,25,87,99,97,91,19
Bloodborne,90,100,45,100,96,98,100,6
Sekiro: Shadows Die Twice,180,94,50,100,94,79,96,18
Elden Ring,240,81,60,100,87,85,78,16


In [13]:
boundries = pd.read_csv("data/boundries.csv", index_col="Boundries")
boundries

,Price,Difficulty,Playthrough time,Graphics,Lore,Soundtrack,Gameplay,Bugs
Boundries,,,,,,,,
b1,151.0,19.0,19.0,17.0,22.0,22.0,22.0,74.0
b2,113.0,40.0,36.0,34.0,43.0,45.0,46.0,56.0
b3,74.0,60.0,53.0,53.0,65.0,67.0,70.0,37.0
b4,37.0,81.0,70.0,71.0,86.0,89.0,92.0,19.0


In [14]:
thresholds = pd.read_csv("data/thresholds.csv", index_col=0, header=[0, 1])
thresholds

Criteria   Price             Difficulty           Playthrough time             \
Thresholds     q     p     v          q    p    v                q    p     v   
Boundries                                                                       
b1          16.0  20.0  26.0        1.0  5.0  8.0              1.0  6.0  12.0   
b2          19.0  22.0  25.0        1.0  5.0  9.0              1.0  6.0   9.0   
b3          17.0  22.0  25.0        1.0  4.0  7.0              1.0  7.0  11.0   
b4          19.0  22.0  28.0        1.0  6.0  9.0              3.0  6.0  12.0   

Criteria   Graphics  ...  Lore Soundtrack            Gameplay              \
Thresholds        q  ...     v          q    p     v        q     p     v   
Boundries            ...                                                    
b1              1.0  ...  10.0        2.0  6.0  10.0      2.0   7.0  11.0   
b2              1.0  ...  13.0        2.0  6.0  12.0      4.0  10.0  14.0   
b3              2.0  ...  10.0        1.0  7.0  11.0      4.0   7.0  12.0   
b4              3.0  ...  14.0        2.0  6.0   9.0      1.0   4.0  10.0   

Criteria   Bugs              
Thresholds    q     p     v  
Boundries                    
b1          4.0  10.0  13.0  
b2          4.0   9.0  13.0  
b3          3.0   7.0  11.0  
b4          3.0   6.0  11.0  

[4 rows x 24 columns]

In [15]:
def marginal_concordances(alternatives, p, q, boundries, types):
    marginal_matrix = np.zeros(alternatives.shape)
    for i, alternative in enumerate(alternatives):
        
        for j in range(len(alternative)):
            
            if types[j] == "gain":
                value = concordance_gain(alternative[j], boundries[j], p[j], q[j])
            else:
                value = concordance_cost(alternative[j], boundries[j], p[j], q[j])
            marginal_matrix[i, j] = value
    return marginal_matrix[np.newaxis, ...]
            

In [16]:
def run_marginal_concordances(alternatives, df_p_q, boundries, types):
    
    composition_matrix = None
    alternatives = alternatives.to_numpy()
    for i in range(len(boundries)):
        p = df_p_q.loc[:, (slice(None), "p")].iloc[i].to_numpy()
        q = df_p_q.loc[:, (slice(None), "q")].iloc[i].to_numpy()
        boundry = boundries.iloc[i].to_numpy()
        matrix = marginal_concordances(alternatives=alternatives, p=p, q=q, boundries=boundry, types=types["type"].to_numpy())
        if i == 0:
            composition_matrix = matrix
        else:
            composition_matrix = np.concatenate((composition_matrix, matrix))
    
    return composition_matrix

In [17]:
test_df_inner = pd.read_csv("data/p_q_correction.csv", index_col="Criteria")

In [18]:
## Numpy implementation
def marginal_concordances_vectorized(alternatives, np_p_q, boundries, types, i):
    end_result = np.zeros(shape=alternatives.shape)

    boundries = np.array(boundries.iloc[i])
    boundries_matrix = np.tile(boundries, (len(alternatives), 1))

    p = np_p_q[0][i]
    q = np_p_q[1][i]

    p_vectorized = np.tile(p, (len(alternatives), 1))
    q_vectorized = np.tile(q, (len(alternatives), 1))
    
    for function_type in ("gain", "cost"):
        gain_type = types == function_type

        test = end_result[:, np.where(gain_type)[0]]

        

        
        if function_type == "gain":
            op1 = (alternatives[:, gain_type] - boundries_matrix[:, gain_type]) >= -q_vectorized[:, gain_type]
            op2 = (alternatives[:, gain_type] - boundries_matrix[:, gain_type]) <  -p_vectorized[:, gain_type]
        else:
            op1 = (alternatives[:, gain_type] - boundries_matrix[:, gain_type]) <= q_vectorized[:, gain_type]
            op2 = (alternatives[:, gain_type] - boundries_matrix[:, gain_type]) > p_vectorized[:, gain_type]
        
        op3 = ~op1 & ~op2
        
        
        test[op1] = 1
        test[op2] = 0

        
        if function_type == "gain":
            test[op3] = ( p_vectorized[:, gain_type][op3] - (boundries_matrix[:, gain_type][op3] - alternatives[:, gain_type][op3]) ) / (p_vectorized[:, gain_type][op3] - q_vectorized[:, gain_type][op3])
        else:
            test[op3] = ( p_vectorized[:, gain_type][op3] - (alternatives[:, gain_type][op3] - boundries_matrix[:, gain_type][op3]) ) / (p_vectorized[:, gain_type][op3] - q_vectorized[:, gain_type][op3])
    
        end_result[:, np.where(gain_type)[0]] = test
    
    return end_result[np.newaxis, ...]

    
    

In [19]:
## Numpy implementation
def marginal_concordances_vectorized_v2(alternatives, np_p_q, boundries, types, ab=True):
    ## I belive that now it is far more flexible and it should not consume that much memory
    ## because now we do not create those huge matrices(relatively huge) but we simply make use of numpy's broadcast capabilities
    
    
    # alternatives = np.tile(alternatives.to_numpy(), (4,1)).reshape((4, 17, 8))
    
    # boundries_matrix = np.tile(boundries.to_numpy(), (1, alternatives.shape[1])).reshape((4, 17, 8))

    # end_result = np.zeros(shape=alternatives.shape)

    p = np_p_q[0]
    q = np_p_q[1]

    # p_vectorized = np.tile(p, (1, alternatives.shape[1])).reshape((4, 17, 8))
    # q_vectorized = np.tile(q, (1, alternatives.shape[1])).reshape((4, 17, 8))

    ##number of boundries
    nob = len(boundries)
    ##number of alternatives
    noa = len(alternatives)
    ##number of functions
    nof = len(p[0])
    if ab:
        alternatives = alternatives.to_numpy()[np.newaxis, ...]
        boundries_matrix = boundries.to_numpy()[:, np.newaxis, :]
    else:
        boundries_matrix = alternatives.to_numpy()[np.newaxis, :, :]
        alternatives = boundries.to_numpy()[:, np.newaxis, :]
        
    
    p_vectorized = p[:, np.newaxis, :]
    q_vectorized = q[:, np.newaxis, :]

    
    end_result = np.zeros(shape=(nob, noa, nof))

    
  
    for function_type in ("gain", "cost"):
        gain_type = types == function_type

        test = end_result[..., np.where(gain_type)[0]]

        

        
        if function_type == "gain":
            # print(gain_type.shape, alternatives.shape, q_vectorized.shape)
            op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) >= -q_vectorized[..., gain_type]
            op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <  -p_vectorized[..., gain_type]
        else:
            op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <= q_vectorized[..., gain_type]
            op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) > p_vectorized[..., gain_type]
        
        op3 = ~op1 & ~op2
        
        
        test[op1] = 1
        test[op2] = 0
        
        if function_type == "gain":
            test[op3] = ( ( p_vectorized[..., gain_type] - (boundries_matrix[..., gain_type] - alternatives[..., gain_type]) ) / (p_vectorized[..., gain_type] - q_vectorized[..., gain_type]) )[op3]
        else:
            test[op3] = ( ( p_vectorized[..., gain_type] - (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) ) / (p_vectorized[..., gain_type] - q_vectorized[..., gain_type]) )[op3]
        
        end_result[..., np.where(gain_type)[0]] = test
    
    return end_result

    
    

In [20]:
def marginal_discordances_vectorized(alternatives, n_p_v, boundries, types, ab=True):
    ## I belive that now it is far more flexible and it should not consume that much memory
    ## because now we do not create those huge matrices(relatively huge) but we simply make use of numpy's broadcast capabilities
    
    
    # alternatives = np.tile(alternatives.to_numpy(), (4,1)).reshape((4, 17, 8))
    
    # boundries_matrix = np.tile(boundries.to_numpy(), (1, alternatives.shape[1])).reshape((4, 17, 8))

    # end_result = np.zeros(shape=alternatives.shape)

    p = n_p_v[0]
    v = n_p_v[1]

    # p_vectorized = np.tile(p, (1, alternatives.shape[1])).reshape((4, 17, 8))
    # q_vectorized = np.tile(q, (1, alternatives.shape[1])).reshape((4, 17, 8))

    ##number of boundries
    nob = len(boundries)
    ##number of alternatives
    noa = len(alternatives)
    ##number of functions
    nof = len(p[0])
    if ab:
        alternatives = alternatives.to_numpy()[np.newaxis, ...]
        boundries_matrix = boundries.to_numpy()[:, np.newaxis, :]
    else:
        boundries_matrix = alternatives.to_numpy()[np.newaxis, :, :]
        alternatives = boundries.to_numpy()[:, np.newaxis, :]
        
    
    p_vectorized = p[:, np.newaxis, :]
    v_vectorized = v[:, np.newaxis, :]

    
    end_result = np.zeros(shape=(nob, noa, nof))

    
  
    for function_type in ("gain", "cost"):
        gain_type = types == function_type

        test = end_result[..., np.where(gain_type)[0]]

        

        
        if function_type == "gain":
            # print(gain_type.shape, alternatives.shape, q_vectorized.shape)
            op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <= -v_vectorized[..., gain_type]
            op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) >= -p_vectorized[..., gain_type]
        else:
            op1 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) >= v_vectorized[..., gain_type]
            op2 = (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) <= p_vectorized[..., gain_type]
        
        op3 = ~op1 & ~op2
        
        
        test[op1] = 1
        test[op2] = 0
        
        if function_type == "gain":
            test[op3] = ( ( (boundries_matrix[..., gain_type] - alternatives[..., gain_type]) - p_vectorized[..., gain_type] ) / (v_vectorized[..., gain_type] - p_vectorized[..., gain_type]) )[op3]
        else:
            test[op3] = ( ( v_vectorized[..., gain_type] - (alternatives[..., gain_type] - boundries_matrix[..., gain_type]) ) / (v_vectorized[..., gain_type] - p_vectorized[..., gain_type]) )[op3]
        
        end_result[..., np.where(gain_type)[0]] = test
    
    return end_result
    

In [21]:
ps = thresholds.loc[:, (slice(None), "p")].to_numpy()[np.newaxis, ...]
vs = thresholds.loc[:, (slice(None), "v")].to_numpy()[np.newaxis, ...]
np_p_v = np.concatenate([ps, vs])
np_p_v

array([[[20.,  5.,  6.,  4.,  5.,  6.,  7., 10.],
        [22.,  5.,  6.,  6.,  9.,  6., 10.,  9.],
        [22.,  4.,  7.,  6.,  6.,  7.,  7.,  7.],
        [22.,  6.,  6.,  7.,  8.,  6.,  4.,  6.]],

       [[26.,  8., 12., 10., 10., 10., 11., 13.],
        [25.,  9.,  9., 12., 13., 12., 14., 13.],
        [25.,  7., 11.,  9., 10., 11., 12., 11.],
        [28.,  9., 12., 11., 14.,  9., 10., 11.]]])

In [22]:

qs = thresholds.loc[:, (slice(None), "q")].to_numpy()[np.newaxis, ...]
pqs = np.concatenate([ps, qs])
pqs.shape

(2, 4, 8)

In [23]:
weights = test_df_inner["weights"].to_numpy()[np.newaxis, np.newaxis, :]
weights

array([[[ 5,  6,  3,  4,  9, 10, 10,  4]]])

In [24]:
marginal_concordances_vectorized_v2(alternatives, pqs, boundries, test_df_inner["type"])[0]

array([[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 0.4, 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ]])

In [25]:
def outranking_credibility(marginal_concordances_tensor, marginal_discordances_tensor, weights):
    final_concordances = (marginal_concordances_tensor * (weights / weights.sum())).sum(axis=2)[..., np.newaxis]
    products = marginal_discordances_tensor > final_concordances

    
    test = np.zeros(marginal_discordances_tensor.shape)
    test1 = np.ones(marginal_discordances_tensor.shape)
    
    test[products] = marginal_discordances_tensor[products]
    
    test1[products] = ( (1 - test) / (1 - final_concordances) )[products]
    test2 = test1.prod(axis=2)[..., np.newaxis]
    # print(test1[3].round(2))
    # print(test2[3])
    
    return test2

In [26]:
marginal_concordances_tensor = marginal_concordances_vectorized_v2(alternatives, pqs, boundries, test_df_inner["type"])
marginal_discordances_tensor = marginal_discordances_vectorized(alternatives, np_p_v, boundries, test_df_inner["type"], ab=True)

In [27]:
marginal_concordances_tensor1 = marginal_concordances_vectorized_v2(alternatives, pqs, boundries, test_df_inner["type"], ab=False)
marginal_discordances_tensor1 = marginal_discordances_vectorized(alternatives, np_p_v, boundries, test_df_inner["type"], ab=False)

In [28]:
as_outranking = outranking_credibility(marginal_concordances_tensor, marginal_discordances_tensor, weights)

/tmp/ipykernel_44091/3586792473.py:11: RuntimeWarning: divide by zero encountered in true_divide
  test1[products] = ( (1 - test) / (1 - final_concordances) )[products]


In [29]:
bs_outranking = outranking_credibility(marginal_concordances_tensor1, marginal_discordances_tensor1, weights)

/tmp/ipykernel_44091/3586792473.py:11: RuntimeWarning: divide by zero encountered in true_divide
  test1[products] = ( (1 - test) / (1 - final_concordances) )[products]


In [30]:
def create_symbols(as_outranking, bs_outranking, sigma=0.65):

    concatenated = np.concatenate([as_outranking, bs_outranking], axis=2)

    utility = np.where(concatenated >= sigma, 1, 0)
    
    signs = np.empty(shape=concatenated.shape[:-1], dtype=str)

    boolean_better = utility[:, :, 0] > utility[:, :, 1]
    boolean_worse = utility[:, :, 0] < utility[:, :, 1]
    boolean_indifferent = (utility[:, :, 0] == utility[:, :, 1]) & (utility[:, :, 0] == 1)
    boolean_incomparable = (utility[:, :, 0] == utility[:, :, 1]) & (utility[:, :, 0] == 0)

    signs[boolean_better] = ">"
    signs[boolean_worse] = "<"
    signs[boolean_indifferent] = "/"
    signs[boolean_incomparable] = "?"
    
    return signs.T
    
signs = create_symbols(as_outranking, bs_outranking)

In [31]:
signs

array([['>', '>', '?', '<'],
       ['>', '>', '?', '?'],
       ['>', '>', '?', '<'],
       ['>', '?', '?', '?'],
       ['>', '?', '?', '?'],
       ['>', '?', '?', '?'],
       ['>', '>', '>', '?'],
       ['?', '?', '?', '?'],
       ['?', '?', '?', '?'],
       ['>', '>', '?', '?'],
       ['>', '>', '?', '?'],
       ['>', '>', '?', '?'],
       ['>', '>', '?', '?'],
       ['>', '>', '?', '?'],
       ['>', '>', '?', '<'],
       ['>', '>', '<', '<'],
       ['>', '>', '?', '?'],
       ['>', '>', '>', '>']], dtype='<U1')

In [32]:
def optimistic(signs):
    return pd.DataFrame(data=signs).isin((">", "/", "?")).sum(axis=1) + 1
def not_optimistic(signs):
    return (~pd.DataFrame(data=signs).isin(("<", "?"))).sum(axis=1) + 1
not_optimistic(signs)

0     3
1     3
2     3
3     2
4     2
5     2
6     4
7     1
8     1
9     3
10    3
11    3
12    3
13    3
14    3
15    3
16    3
17    5
dtype: int64

In [33]:
def run_marginal_concordances_vectorized(alternatives, thresholds, boundries, types):
    final_matrix = None
    al_before = alternatives.to_numpy()
    ps = thresholds.loc[:, (slice(None), "p")].to_numpy()
    qs = thresholds.loc[:, (slice(None), "q")].to_numpy()
    pqs = np.concatenate([ps[np.newaxis, ...], qs[np.newaxis, ...]], axis=0)
    for i in range(pqs.shape[1]):
        matrix = marginal_concordances_vectorized(al_before, pqs, boundries, types, i)
        if final_matrix is None:
            final_matrix = matrix
        else:
            final_matrix = np.concatenate([final_matrix, matrix])
    return final_matrix
        

In [34]:
run_marginal_concordances_vectorized(alternatives, thresholds, boundries, test_df_inner["type"])[1]

array([[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.6],
       [0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ]])

In [35]:
def concordance_gain(a, b, p, q):
    if a - b >= -q:
        return 1
    elif a - b < -p:
        return 0
    else:
        value = (p - (b - a)) / (p - q)
        return value

def concordance_cost(a, b, p, q):
    if a - b <= q:
        return 1
    elif a - b > p:
        return 0
    else:
        value = (p - (a - b)) / (p - q)
        return value

In [36]:
def discordance_gain(a, b, p, v):
    if a - b <= -v:
        return 1
    elif a - b >= p:
        return 0
    else:
        value = ((b - a) - p) / (v - p)
        return value

def discordance_cost(a, b, p, v):
    if a - b >= v:
        return 1
    elif a - b <= p:
        return 0
    else:
        value = (v - (a - b)) / (v - p)
        return value

In [37]:
run_marginal_concordances(alternatives, thresholds, boundries, test_df_inner)[2].round(2)

array([[1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.67, 1.  , 1.  , 1.  , 0.  , 0.  ],
       [0.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.  , 1.  , 0.67, 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.  , 1.  , 0.  , 1.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.67, 1.  , 1.  , 0.  , 1.  , 1.  ],
       [1.  , 0.67, 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 0.  , 0.67, 1.  , 1.  , 1.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 1.  , 1.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 1.  , 1.  ,

### Conclusions

In [38]:
(run_marginal_concordances(alternatives, thresholds, boundries, test_df_inner) == run_marginal_concordances_vectorized(alternatives, thresholds, boundries, test_df_inner["type"])).all()

True

In [39]:
(run_marginal_concordances_vectorized(alternatives, thresholds, boundries, test_df_inner["type"]) == marginal_concordances_vectorized_v2(alternatives, pqs, boundries, test_df_inner["type"])).all()

True

In [40]:
(marginal_concordances_vectorized_v2(alternatives, pqs, boundries, test_df_inner["type"]) == run_marginal_concordances(alternatives, thresholds, boundries, test_df_inner)).all()

True

In [41]:
## I am fully aware that those tests have transitive relationship and it is enough to do only 2 comparisons instead of one
## that's because if a == b and b == c, then it follows that a == c

In [42]:
%%timeit
run_marginal_concordances_vectorized(alternatives, thresholds, boundries, test_df_inner["type"])

3.12 ms ± 60.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%timeit
run_marginal_concordances(alternatives, thresholds, boundries, test_df_inner)

7.76 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
marginal_concordances_vectorized_v2(alternatives, pqs, boundries, test_df_inner["type"])

401 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The numpy vectorized solution using broadcasting is approximately 20 times faster than the regular one